# Beginning Stuff

In [ ]:
!pip install -Uqq fastai

In [ ]:
from fastai.vision.all import *
from fastai.text.all import *

In [ ]:
import pandas as pd
import numpy as np
from fastai.tabular.all import *

In [ ]:
!unzip ../input/movie-review-sentiment-analysis-kernels-only/test.tsv.zip -d test
!unzip ../input/movie-review-sentiment-analysis-kernels-only/train.tsv.zip -d train

In [ ]:
#kernals only = use the kaggle version of notebooks (like their version of colab)
df_train = pd.read_csv("train/train.tsv", sep='\t', header=0, low_memory=False) #tell them that tab separaters + tell them it has headers
df_test = pd.read_csv("test/test.tsv", sep='\t', header=0, low_memory=False)

#look more carefully at the problem description and see why they're giving us phrases and such

In [ ]:
df_train_sample = df_train.sample(frac = 1.0)
df_test_sample = df_test.sample(frac = 1.0)

In [ ]:
df_combined_sample = pd.concat([df_train_sample, df_test_sample], ignore_index=True)

In [ ]:
del df_train_sample['PhraseId']
del df_train_sample['SentenceId']

In [ ]:
df_test_sample

# Train

Language model stuff

In [ ]:
dls_lm = TextDataLoaders.from_df(df_train_sample, label_col = 1, text_col = 0, valid_pct=0.1, is_lm=True, seed = 5)

In [ ]:
learn = language_model_learner(dls_lm, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

In [ ]:
lr_para = learn.lr_find(num_it=100)
min_grad_lr = lr_para.lr_min
print(min_grad_lr)

In [ ]:
learn.fit_one_cycle(5, lr_max = min_grad_lr, cbs = SaveModelCallback()) 

In [ ]:
learn.save_encoder('finetuned')

In [ ]:
dls = TextDataLoaders.from_df(df_train_sample, label_col = 1, text_col = 0, valid_pct=0.1, is_lm=False, seed = 5, bs = 4)

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, metrics=accuracy, cbs = SaveModelCallback)
learn = learn.load_encoder('finetuned')

In [ ]:
learn.fit_one_cycle(1, 2e-2)

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, slice(1e-3/(2.6**4),1e-3))

In [ ]:
prediction_list_model = []

for i in range(len(df_test_sample)):
  sentiment = learn.predict(df_test_sample.iloc[i]['Phrase'])[0]
  prediction_list_model.append(sentiment)

In [ ]:
df_test_results = pd.DataFrame({'PhraseID': df_test_sample['PhraseId'], 'Sentiment': prediction_list_model})

In [ ]:
df_test_results.to_csv('./submission.csv', index=False)

#FOR SUBMISSION:
#https://www.kaggle.com/dansbecker/submitting-from-a-kernel

# Test